In [1]:
import math
import os
import uuid
import time

from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.patheffects # For text outlines
import numpy as np
import tensorflow as tf
import traceback # Import for detailed error logging within the function

try:
    from google.protobuf import text_format
    from waymo_open_dataset.protos import scenario_pb2
    from waymo_open_dataset.protos import map_pb2 # For LaneBoundary.Type
    print("Waymo Open Dataset modules imported successfully.")
except ImportError as e:
    print(f"Error importing Waymo modules: {e}")
    print("Please ensure 'waymo-open-dataset' is installed: pip install waymo-open-dataset")
    exit()

INPUT_DIR = '/home/hamdarlab/Desktop/Waymo/data20s/' # Adjust if your path changed
OUTPUT_DIR = '/home/hamdarlab/Desktop/Waymo/data20s/lanes_centers_only/'

try:
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"Output directory '{OUTPUT_DIR}' ensured.")
except OSError as e:
    print(f"Error creating output directory '{OUTPUT_DIR}': {e}")
    exit()

def create_figure_and_axes(size_pixels):
  fig, ax = plt.subplots(1, 1, num=uuid.uuid4())
  dpi = 100; size_inches = size_pixels / dpi
  fig.set_size_inches([size_inches, size_inches]); fig.set_dpi(dpi)
  fig.set_facecolor('white'); ax.set_facecolor('white')
  ax.xaxis.label.set_color('black'); ax.tick_params(axis='x', colors='black')
  ax.yaxis.label.set_color('black'); ax.tick_params(axis='y', colors='black')
  fig.set_tight_layout(True); ax.grid(False)
  return fig, ax

def fig_canvas_image(fig):
  fig.subplots_adjust(left=0.08, bottom=0.08, right=0.98, top=0.98, wspace=0.0, hspace=0.0)
  fig.canvas.draw(); data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  return data.reshape(fig.canvas.get_width_height()[::-1] + (3,))

def get_map_viewport_from_polylines(all_polylines_dict, other_points_xyz_list=None):
    all_map_points_x, all_map_points_y = [], []
    if not all_polylines_dict and not other_points_xyz_list: return 0,0,10
    if all_polylines_dict:
        for polyline_pts in all_polylines_dict.values(): # Iterate directly over values
            if polyline_pts.shape[0] > 0:
                all_map_points_x.extend(polyline_pts[:, 0])
                all_map_points_y.extend(polyline_pts[:, 1])
    if other_points_xyz_list:
        for pt in other_points_xyz_list:
            all_map_points_x.append(pt[0]); all_map_points_y.append(pt[1])
    if not all_map_points_x: return 0,0,10
    min_x,max_x = np.min(all_map_points_x),np.max(all_map_points_x)
    min_y,max_y = np.min(all_map_points_y),np.max(all_map_points_y)
    center_x,center_y = (min_x+max_x)/2,(min_y+max_y)/2
    range_x,range_y = max_x-min_x,max_y-min_y
    map_width = max(range_x,range_y,20.0)
    return center_y,center_x,map_width

def visualize_scenario_static(scenario_proto, size_pixels=1000):
    lanes_data = {}
    all_map_features_polylines = {}
    other_map_feature_points_list = []
    lane_map_features_dict = {}

    if not scenario_proto.map_features:
        # print(f"    Scenario {scenario_proto.scenario_id}: No map features. Skipping.")
        return None

    for map_feature in scenario_proto.map_features:
        feature_id = map_feature.id
        poly_points_list = []
        if map_feature.HasField('lane'):
            lane_map_features_dict[feature_id] = map_feature
            for p in map_feature.lane.polyline: poly_points_list.append((p.x,p.y))
            if poly_points_list: lanes_data[feature_id]=np.array(poly_points_list)
        elif map_feature.HasField('road_line'):
            for p in map_feature.road_line.polyline: poly_points_list.append((p.x,p.y))
        elif map_feature.HasField('road_edge'):
            for p in map_feature.road_edge.polyline: poly_points_list.append((p.x,p.y))
        elif map_feature.HasField('crosswalk'):
            for p in map_feature.crosswalk.polygon: poly_points_list.append((p.x,p.y))
            if poly_points_list: poly_points_list.append(poly_points_list[0])
        elif map_feature.HasField('speed_bump'):
            for p in map_feature.speed_bump.polygon: poly_points_list.append((p.x,p.y))
            if poly_points_list: poly_points_list.append(poly_points_list[0])
        elif map_feature.HasField('driveway'):
            for p in map_feature.driveway.polygon: poly_points_list.append((p.x,p.y))
            if poly_points_list: poly_points_list.append(poly_points_list[0])
        elif map_feature.HasField('stop_sign'):
            pos = map_feature.stop_sign.position
            other_map_feature_points_list.append([pos.x,pos.y,pos.z])
        if poly_points_list:
            all_map_features_polylines[feature_id]=np.array(poly_points_list)
    
    if not lanes_data and not all_map_features_polylines and not other_map_feature_points_list:
        # print(f"    Scenario {scenario_proto.scenario_id}: No plottable map data. Skipping.")
        return None

    fig, ax = create_figure_and_axes(size_pixels=size_pixels)
    center_y, center_x, map_plot_width = get_map_viewport_from_polylines(all_map_features_polylines, other_map_feature_points_list)

    for feature_id, poly_pts in all_map_features_polylines.items():
        if feature_id not in lanes_data:
            if poly_pts.shape[0]>1: ax.plot(poly_pts[:,0],poly_pts[:,1],color='lightgray',linewidth=1,linestyle='-',alpha=0.5,zorder=1)
            elif poly_pts.shape[0]==1: ax.plot(poly_pts[0,0],poly_pts[0,1],'o',color='lightgray',markersize=2,alpha=0.5,zorder=1)
    for pt in other_map_feature_points_list:
        ax.plot(pt[0],pt[1],'s',color='red',markersize=4,alpha=0.6,zorder=1.1)

    lane_id_to_color={}
    if lanes_data:
        lane_ids_sorted=sorted(list(lanes_data.keys()))
        num_unique_lanes=len(lane_ids_sorted)
        if num_unique_lanes>0:
            cmap=plt.cm.get_cmap('nipy_spectral')
            colors_resolved = cmap(np.linspace(0.05,0.95,num_unique_lanes)) if num_unique_lanes > 1 else [cmap(0.5)]
            lane_id_to_color={lid:colors_resolved[i] for i,lid in enumerate(lane_ids_sorted)}
        for lane_id,centerline_pts in lanes_data.items():
            color=lane_id_to_color.get(lane_id,'gray')
            if centerline_pts.shape[0]>1: ax.plot(centerline_pts[:,0],centerline_pts[:,1],color=color,linewidth=1.5,alpha=0.8,zorder=2)

    # --- MODIFIED AND MORE ROBUST BOUNDARY PLOTTING ---
    if lane_map_features_dict:
        for lane_id, lane_feature_proto in lane_map_features_dict.items():
            lane_color = lane_id_to_color.get(lane_id)
            if lane_color is None: continue

            # Defensive check for lane attribute and its boundaries
            if not (hasattr(lane_feature_proto, 'lane') and \
                    lane_feature_proto.lane is not None and \
                    hasattr(lane_feature_proto.lane, 'boundaries')):
                # print(f"    DEBUG: Scenario {scenario_proto.scenario_id}, Lane {lane_id}: 'lane' field or 'boundaries' attribute missing or lane is None.")
                continue
            
            try:
                for i_boundary, boundary_obj in enumerate(lane_feature_proto.lane.boundaries):
                    try:
                        boundary_fid = boundary_obj.boundary_feature_id
                        boundary_polyline = all_map_features_polylines.get(boundary_fid)

                        if boundary_polyline is None or boundary_polyline.shape[0] == 0:
                            # print(f"    DEBUG: Scenario {scenario_proto.scenario_id}, Lane {lane_id}, Boundary {i_boundary}: Boundary feature ID {boundary_fid} not found or has empty polyline.")
                            continue

                        actual_poly_len = boundary_polyline.shape[0]
                        data_start_idx = boundary_obj.boundary_start_index
                        data_end_idx = boundary_obj.boundary_end_index # Waymo doc: inclusive end index for the segment on the boundary feature

                        # Validate indices against the actual length of the boundary feature's polyline
                        if not (0 <= data_start_idx < actual_poly_len and \
                                0 <= data_end_idx < actual_poly_len and \
                                data_start_idx <= data_end_idx):
                            # print(f"    DEBUG: Scenario {scenario_proto.scenario_id}, Lane {lane_id}, Boundary {i_boundary} (feat_id:{boundary_fid}): "
                            #       f"Invalid raw indices (start={data_start_idx}, end={data_end_idx}) for polyline of length {actual_poly_len}.")
                            continue
                            
                        # Slice using validated indices. End index for slice needs to be +1 as Python slicing is exclusive.
                        segment_to_plot = boundary_polyline[data_start_idx : data_end_idx + 1]
                        
                        if segment_to_plot.shape[0] > 1: # Need at least two points to plot a line
                            linestyle = '-'
                            # Example: Modify linestyle based on boundary_type
                            # if boundary_obj.boundary_type == map_pb2.LaneBoundary.ROAD_LINE_TYPE_BROKEN_SINGLE_WHITE:
                            #    linestyle = '--'
                            ax.plot(segment_to_plot[:, 0], segment_to_plot[:, 1],
                                    color=lane_color, linewidth=2.0, 
                                    linestyle=linestyle, alpha=0.85, zorder=2.5)
                        # else:
                            # print(f"    DEBUG: Scenario {scenario_proto.scenario_id}, Lane {lane_id}, Boundary {i_boundary}: Segment too short (shape {segment_to_plot.shape})")

                    except AttributeError as ae_b:
                        print(f"    WARNING: AttributeError in single boundary processing (Scenario {scenario_proto.scenario_id}, Lane {lane_id}, Boundary Index {i_boundary}): {ae_b}")
                        # traceback.print_exc(limit=1)
                        continue 
                    except IndexError as ie_b:
                        print(f"    WARNING: IndexError in single boundary processing (Scenario {scenario_proto.scenario_id}, Lane {lane_id}, Boundary Index {i_boundary}): {ie_b}")
                        # traceback.print_exc(limit=1)
                        continue
                    except Exception as e_b: # Catch any other error for a single boundary
                        print(f"    WARNING: Unexpected error processing single boundary (Scenario {scenario_proto.scenario_id}, Lane {lane_id}, Boundary Index {i_boundary}): {e_b}")
                        # traceback.print_exc(limit=1)
                        continue
            except Exception as e_lane_b: # Catch error iterating boundaries for a whole lane
                 print(f"    WARNING: Error iterating boundaries for lane (Scenario {scenario_proto.scenario_id}, Lane {lane_id}): {e_lane_b}")
                 # traceback.print_exc(limit=1) # Limit traceback to avoid too much console spam
                 continue # Skip processing boundaries for this entire lane
    # --- END OF BOUNDARY PLOTTING ---

    if lanes_data: # Plot Lane ID Text
        for lane_id, centerline_pts in lanes_data.items():
            if centerline_pts.shape[0]>0:
                text_pos_idx=centerline_pts.shape[0]//2
                ax.text(centerline_pts[text_pos_idx,0],centerline_pts[text_pos_idx,1],f'{lane_id}',
                        color='black',fontsize=4.5,ha='center',va='center',zorder=3,
                        path_effects=[matplotlib.patheffects.withStroke(linewidth=0.75,foreground='white')])

    ax.set_title(f'Scenario ID: {scenario_proto.scenario_id} - Lanes and Boundaries',fontsize=10)
    ax.axis([-map_plot_width/2+center_x,map_plot_width/2+center_x,-map_plot_width/2+center_y,map_plot_width/2+center_y])
    ax.set_aspect('equal'); ax.set_xlabel("X coordinate (m)"); ax.set_ylabel("Y coordinate (m)")
    image=fig_canvas_image(fig); plt.close(fig)
    return image

# --- Main script (ensure traceback.print_exc() is active in the except block as shown in Step 1) ---
# ... (rest of your main script) ...
# (Make sure the main script uses the visualize_scenario_static function above)


# --- Main script to process all files (remains the same structure) ---
print(f"\n--- Starting Plot Generation (Lanes Only) ---")
print(f"Input directory: {INPUT_DIR}")
print(f"Plots will be saved to: {OUTPUT_DIR}")

tfrecord_files = []
for root, _, files in os.walk(INPUT_DIR):
    for f in files:
        if '.tfrecord' in f: tfrecord_files.append(os.path.join(root, f))
tfrecord_files.sort()

if not tfrecord_files:
    print(f"\nERROR: No TFRecord files found in '{INPUT_DIR}'.")
else:
    print(f"\nFound {len(tfrecord_files)} TFRecord files to process.")
    for k, found_file in enumerate(tfrecord_files[:min(5, len(tfrecord_files))]):
        print(f"  Preview file {k+1}: {os.path.basename(found_file)}")
    if len(tfrecord_files) > 5: print("  ...")

    for i, full_filepath in enumerate(tfrecord_files):
        filename = os.path.basename(full_filepath)
        base_output_name = os.path.splitext(filename)[0]
        output_filepath_prefix = os.path.join(OUTPUT_DIR, base_output_name)

        print(f"\n[{i+1}/{len(tfrecord_files)}] Processing '{filename}'...")
        try:
            dataset = tf.data.TFRecordDataset(full_filepath, compression_type='')
            successful_plots_in_file = 0
            scenarios_in_file = 0
            for j, data_bytes in enumerate(dataset.as_numpy_iterator()):
                scenarios_in_file +=1
                scenario_id_for_log = "N/A"
                try:
                    scenario = scenario_pb2.Scenario(); scenario.ParseFromString(data_bytes)
                    scenario_id_for_log = scenario.scenario_id
                    
                    plot_image = visualize_scenario_static(scenario) # Calls the updated function
                    
                    if plot_image is None or plot_image.size == 0: continue

                    scenario_output_filepath = f"{output_filepath_prefix}_scenario_{scenario_id_for_log}_lanes_only.png"
                    plt.imsave(scenario_output_filepath, plot_image)
                    successful_plots_in_file += 1
                except Exception as e:
                    print(f"  - WARNING: Error processing scenario {j} (ID: {scenario_id_for_log}) in '{filename}': {e}")
                    # import traceback; traceback.print_exc() # For debugging
                    continue
            
            if scenarios_in_file == 0: print(f"  - No scenarios found in '{filename}'.")
            elif successful_plots_in_file == 0: print(f"  - Processed {scenarios_in_file} scenarios in '{filename}', none plotted.")
            else: print(f"  - Plotted {successful_plots_in_file}/{scenarios_in_file} map-only scenarios from '{filename}'.")

        except tf.errors.DataLossError as e: print(f"  - ERROR: Data loss in '{filename}': {e}. Skipping.")
        except Exception as e: print(f"  - ERROR: General error with '{filename}': {e}. Skipping.")
            # import traceback; traceback.print_exc() # For debugging

print("\n--- Plotting process complete. ---")
print(f"Check '{OUTPUT_DIR}' for generated images.")

2025-05-24 14:57:59.279139: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-24 14:57:59.317706: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-24 14:57:59.318391: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-24 14:58:00.061686: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Waymo Open Dataset modules imported successfully.
Output directory '/home/hamdarlab/Desktop/Waymo/data20s/lanes_centers_only/' ensured.

--- Starting Plot Generation (Lanes Only) ---
Input directory: /home/hamdarlab/Desktop/Waymo/data20s/
Plots will be saved to: /home/hamdarlab/Desktop/Waymo/data20s/lanes_centers_only/

Found 20 TFRecord files to process.
  Preview file 1: training_20s.tfrecord-00000-of-01000
  Preview file 2: training_20s.tfrecord-00001-of-01000
  Preview file 3: training_20s.tfrecord-00002-of-01000
  Preview file 4: training_20s.tfrecord-00003-of-01000
  Preview file 5: training_20s.tfrecord-00004-of-01000
  ...

[1/20] Processing 'training_20s.tfrecord-00000-of-01000'...


2025-05-24 14:58:01.328579: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

KeyboardInterrupt



Error in callback <function flush_figures at 0x7f37e860a560> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt

